In [4]:
import torch
import numpy as np
import torch.nn as nn
Deg2Rad = np.pi/180

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(5, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 40),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(40, 3)
        )

    def forward(self, x):
        x=x.view(1,-1)
        out=self.fc(x)
        return out

In [6]:
tr_model=nn.DataParallel(Model().cuda())

In [8]:
tr_model.load_state_dict(torch.load("train_res_fin"))
tr_model.eval()
mean=np.load('mean.npy')
std=np.load('std.npy')

In [4]:
g=9.8
K_alt=1.0
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad
dist_sep = 100
ac = 0
r= 1000
vc = 100
elev = 0
azim = 0
hdot_cmd = 0

los = 0
los_p = los
los_p2 = los_p

dlos = 0
dlos_p = dlos
dlos_p2 = dlos_p

azim = 0
azim_p = azim
azim_p2 = azim_p

daz = 0
daz_p = daz
daz_p2 = daz_p

ATAR_update_couter_p = 0

# received packet number
packetCounter = 0

# aircraft & sensor models update state counters, increases by 1 each time the model updates it's state
RADALT_update_couter = 0
ATAR_update_couter = 0
ADSB_update_counter = 0
OWNship_update_counter =0
Intruder_update_counter = 0
Frame_update_counter =0

# radio altimeter data (sensor) _ OWN-ship altitude
RADALT_vertical_rate = 0
RADALT_ground_altitude = 0

# Air_to_Air_Radar data (sensor)
ATAR_intruder_bearing =0
ATAR_intruder_elevation = 0
ATAR_intruder_closing_velocity = 0
ATAR_intruder_relative_distance = 100000
ATAR_intruder_detection_flag = False

# ADSB data (sensor)
ADSB_intruder_longitude = 0
ADSB_intruder_latitude = 0
ADSB_intruder_altitude = 0
ADSB_intruder_heading = 0
ADSB_intruder_velocity = 0
ADSB_intruder_vertical_rate =0
ADSB_intruder_callsign = " "
ADSB_intruder_iCAOadd = " "

# OWN-ship states (dynamic model)
OWNship_longitude =0
Ownship_latitude =0
OWNship_altitude =0
OWNship_heading =0
OWNship_velocity =0
OWNship_pos_x =0
OWNship_pos_y=0
OWNship_pos_z=0
OWNship_roll =0
OWNship_pitch =0
OWNship_yaw =0
OWNship_vertical_rate =0

# INTruder states (dynamic model)
INTruder_longitude =0
INTruderlatitude =0
INTruderaltitude =0
INTruder_heading =0
INTruder_velocity =0
INTruder_pos_x =0
INTruder_pos_y=0
INTruder_pos_z=0
INTruder_roll =0
INTruder_pitch =0
INTruder_yaw =0
INTruder_vertical_rate =0

#############################################

######## packet ICD (python to unity) #######

header1 = 80
header2 = 85
header3 = 67

flightMode = 1   #mode 1 = auto , mode 0 = manual

rollCMD = 0   #0~100~200 (roll rate)
alphaCMD = 0  #0~100~200 (pitch rate)
thrustCMD = 0  #0~100~200 (0 being 0 thrust and 200 full thrust )

PanCMD = 0
TiltCMD = 0
lockModeCMD = 0
ZominCMD = 0
ZomoutCMD = 0
cameraLselect =1
cameraRselect = 0



cmdPacket = bytearray(7)
cameraCmdPacket = bytearray(10)


def setCMD(mode,roll, alpha, thrust,DBPAStakeOver):
    cmdPacket[0] = header1
    cmdPacket[1] = header2
    cmdPacket[2] = mode
    cmdPacket[3] = roll
    alpha=int(alpha)
    cmdPacket[4] = alpha
    cmdPacket[5] = thrust
    cmdPacket[6] = DBPAStakeOver
    sock.sendto(cmdPacket,('163.180.116.56', 11000))

In [ ]:
import socket
results=[]
sock=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(("163.180.116.46",55666))
while True:
    received_data, addr = sock.recvfrom(1024)
    
    SILS_stat =[]
    deliminatorindex = -1

    for char in range(0,len(received_data)):
        if received_data[char] == 32:
            SILS_stat.append(received_data[deliminatorindex+1:char])
            deliminatorindex = char
    
    RADALT_update_couter = float(SILS_stat[1])
    ATAR_update_couter =float(SILS_stat[2])
    ADSB_update_counter = float(SILS_stat[3])
    OWNship_update_counter = float(SILS_stat[4])
    Intruder_update_counter = float(SILS_stat[5])
    Frame_update_counter = float(SILS_stat[6])

    RADALT_vertical_rate = float(SILS_stat[7])
    RADALT_ground_altitude = float(SILS_stat[8])

    ATAR_intruder_bearing = float(SILS_stat[9])
    ATAR_intruder_elevation = float(SILS_stat[10])
    ATAR_intruder_relative_distance = float(SILS_stat[11])
    ATAR_intruder_closing_velocity = float(SILS_stat[11+1])
    ATAR_intruder_detection_flag = SILS_stat[12+1]

    ADSB_intruder_longitude = float(SILS_stat[13+1])
    ADSB_intruder_latitude = float(SILS_stat[14+1])
    ADSB_intruder_altitude = float(SILS_stat[15+1])
    ADSB_intruder_heading = float(SILS_stat[16+1])
    ADSB_intruder_velocity = float(SILS_stat[17+1])
    ADSB_intruder_vertical_rate = float(SILS_stat[18+1])
    ADSB_intruder_callsign =SILS_stat[19+1]
    ADSB_intruder_iCAOadd = SILS_stat[20+1]

    OWNship_longitude = float(SILS_stat[21+1])
    Ownship_latitude = float(SILS_stat[22+1])
    OWNship_altitude = float(SILS_stat[23+1])
    OWNship_heading = float(SILS_stat[24+1])
    OWNship_velocity = float(SILS_stat[25+1])
    OWNship_pos_x =float( SILS_stat[26+1])
    OWNship_pos_y = float(SILS_stat[27+1])
    OWNship_pos_z = float(SILS_stat[28+1])
    OWNship_roll = float(SILS_stat[29+1])
    OWNship_pitch = float(SILS_stat[30+1])
    OWNship_yaw =float( SILS_stat[31+1])
    OWNship_vertical_rate = float(SILS_stat[32+1])

    INTruder_longitude = float(SILS_stat[33+1])
    INTruderlatitude = float(SILS_stat[34+1])
    INTruderaltitude = float(SILS_stat[35+1])
    INTruder_heading = float(SILS_stat[36+1])
    INTruder_velocity = float(SILS_stat[37+1])
    INTruder_pos_x = float(SILS_stat[38+1])
    INTruder_pos_y = float(SILS_stat[39+1])
    INTruder_pos_z = float(SILS_stat[40+1])
    INTruder_roll = float(SILS_stat[41+1])
    INTruder_pitch = float(SILS_stat[42+1])
    INTruder_yaw = float(SILS_stat[43+1])
    INTruder_vertical_rate = float(SILS_stat[44+1])


    


    
    gamma = np.arcsin(OWNship_vertical_rate/max(1,OWNship_velocity,key=abs))
    theta = OWNship_pitch*Deg2Rad
    r = ATAR_intruder_relative_distance 
    vc = -ATAR_intruder_closing_velocity
    elev = ATAR_intruder_elevation*Deg2Rad
    azim = -ATAR_intruder_bearing*Deg2Rad
    los = -elev
    
    if (ATAR_update_couter != ATAR_update_couter_p):
        dlos = 4.61893764e-01*(los - los_p2) + 1.84295612*dlos_p - 0.852194*dlos_p2   # filtered LOS rate, bw=10, zeta=0.8, dt=0.01
        daz  = 4.61893764e-01*(azim-azim_p2) + 1.84295612*daz_p  - 0.852194*daz_p2    # filtered azim rate, bw=10, zeta=0.8, dt=0.01
#         dlos = 1.20144144e-01*(los - los_p2) + 1.92072072*dlos_p - 0.92312312*dlos_p2   # filtered LOS rate, bw=5, zeta=0.8, dt=0.01
#         daz  = 1.20144144e-01*(azim-azim_p2) + 1.92072072*daz_p  - 0.92312312*daz_p2    # filtered azim rate, bw=5, zeta=0.8, dt=0.01
#        dlos = 1.97077059e-02*(los - los_p2) + 1.96811337*dlos_p - 0.96850704*dlos_p2   # filtered LOS rate, bw=2, zeta=0.8, dt=0.01
#        daz  = 1.97077059e-02*(azim-azim_p2) + 1.96811337*daz_p  - 0.96850704*daz_p2    # filtered azim rate, bw=2, zeta=0.8, dt=0.01
        los_p2 = los_p
        los_p = los

        dlos_p2 = dlos_p
        dlos_p = dlos

        azim_p2 = azim_p
        azim_p = azim

        daz_p2 = daz_p
        daz_p = daz

        ATAR_update_couter_p = ATAR_update_couter

        zem_v = r*r/max(1,vc)*dlos
        zem_h = r*r/max(1,vc)*daz

        crm_v = r*los
    
    if ATAR_intruder_relative_distance < 1500 and ATAR_intruder_relative_distance >dist_sep and abs(elev)<40*Deg2Rad and abs(azim)<40*Deg2Rad :

        thrustCMD = 150 + 100*(51.44-OWNship_velocity)
        thrustCMD = int(np.clip(thrustCMD, 0, 255))

        input_dat=torch.tensor(((np.array([r,vc,los,daz,dlos])
             -mean)/std).astype(np.float32)).cuda()
        output=tr_model(input_dat)
        _, predicted=torch.max(output,1)
        if predicted[0]==0:
            hdot_cmd=0.0
        if predicted[0]==1:
            hdot_cmd=-10.0
        if predicted[0]==2:
            hdot_cmd=10.0
            
        ac=K_alt*(hdot_cmd-OWNship_vertical_rate)+g/(np.cos(gamma))-7.0
        ac = np.clip(ac, -30, 30)     
        setCMD(0,100,ac+100,thrustCMD,True)
        
    else:
        thrustCMD = 150 + 100*(51.44-OWNship_velocity)
        thrustCMD = int(np.clip(thrustCMD, 0, 255))
        hdot_cmd=0
        ac=K_alt*(hdot_cmd-OWNship_vertical_rate)+g/(np.cos(gamma))-7.0
        ac = np.clip(ac, -30, 30)  
        setCMD(0,100,ac+100,thrustCMD,True)
        
    results.append([ATAR_intruder_bearing,ATAR_intruder_elevation,ATAR_intruder_relative_distance,
                        ATAR_intruder_closing_velocity,ATAR_intruder_detection_flag,OWNship_longitude,Ownship_latitude,OWNship_altitude,
                        OWNship_heading,OWNship_velocity,OWNship_pos_x,OWNship_pos_y,OWNship_pos_z,OWNship_roll,OWNship_pitch,OWNship_yaw,
                        OWNship_vertical_rate,gamma,ac,hdot_cmd,zem_v,zem_h,crm_v,daz,dlos])
    
#     print("ac:", ac)
#     print("gamma:", gamma)
#     print("ovr:",float(rec_dat[33]))
#     print("r:",r)
#     print("Send IP:", addr[0])
    

In [6]:
import pandas as pd
index=['bearing','elevation','relative_distance','closing_velocity','detection_flag','longitude','latitude','altitude',
'heading','velocity','pos_x','pos_y','pos_z','roll','pitch','yaw','vertical_rate','gamma','ac','hdot_cmd',"zem_v","zem_h","crm_v","daz","dlos"]
name='hdot_cmd_sim.csv'
df=pd.DataFrame(results[:-1],columns=index)
df.to_csv(name)
print('complete write '+name)
sock.close()

complete write hdot_cmd_sim.csv


In [7]:
df=pd.read_csv('hdot_cmd_sim.csv', index_col=0)
df.plot(subplots=True,legend =True,sharex=False,figsize=(20,140),grid=True,layout=(25,1))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7ef0d42550>],
      dtype=object)

In [ ]:
df["hdot_cmd"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1))
df["vertical_rate"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1))

In [ ]:
df["zem_v"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1),ylim=(-200,200))

In [ ]:
df["dlos"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1),ylim=(-0.2,0.2))

In [ ]:
df["elevation"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1))

In [ ]:
df["pitch"].plot(legend =True,sharex=False,figsize=(20,15),grid=True,layout=(20,1))